In [1]:

%run ../../load_magic/storage.py
%run ../../load_magic/dataframes.py
import urllib.request
import re
from bs4 import BeautifulSoup
%pprint
%who

DATA_FOLDER: ../data/
SAVES_FOLDER: ../saves/
Pretty printing has been turned OFF
BeautifulSoup	 DATA_CSV_FOLDER	 DATA_FOLDER	 ENCODING_TYPE	 SAVES_CSV_FOLDER	 SAVES_FOLDER	 SAVES_PICKLE_FOLDER	 attempt_to_pickle	 example_iterrows	 
get_column_descriptions	 load_csv	 load_dataframes	 load_object	 math	 os	 pd	 pickle	 re	 
save_dataframes	 store_objects	 sys	 urllib	 



# Scraping the Civilization Stats

In [2]:

wiki_url = 'https://ageofempires.fandom.com'
def get_page_soup(page_url):
    with urllib.request.urlopen(page_url) as response:
        page_html = response.read()
    page_soup = BeautifulSoup(page_html, 'html.parser')
    
    return page_soup

In [3]:

def get_content_text_div(page_url):
    page_soup = get_page_soup(page_url)
    content_text_selector = '#mw-content-text'
    content_text_div = page_soup.select(content_text_selector)[0]
    
    return content_text_div

In [4]:

table_df_list = []
summary_box_url = '{}/wiki/Age_of_Empires_II:_The_Age_of_Kings'.format(wiki_url)
print(summary_box_url)
with urllib.request.urlopen(summary_box_url) as response:
    summary_box_html = response.read()
summary_box_soup = BeautifulSoup(summary_box_html, 'html.parser')
content_text_selector = '#mw-content-text'
content_text_div = summary_box_soup.select(content_text_selector)[0]
children_list = list(content_text_div.children)
for child in children_list:
    if child.name == 'table':
        summary_box_table = child
        table_df_list.append(pd.read_html(str(child), header=0)[0])
print(len(table_df_list))

https://ageofempires.fandom.com/wiki/Age_of_Empires_II:_The_Age_of_Kings
5


In [35]:

column_name = 'Civilization'
table_df_list[0].columns = ['Architectural Style', column_name]
table_df_list[0].name = 'Architectural Styles'
table_df_list[1].columns = [column_name, 'Wonder Name']
table_df_list[1].name = 'Wonders'
civilizations_df = pd.merge(left=table_df_list[0], right=table_df_list[1],
                         how='outer', on=column_name)
civilizations_df = civilizations_df.set_index(column_name)
civilizations_df.index.name = column_name

In [6]:

civ_page_url = '{}/wiki/Civilizations_(Age_of_Empires_II)'.format(wiki_url)
civ_content_text_div = get_content_text_div(civ_page_url)

In [59]:

civ_selector = 'span[class="mw-headline"]'
civ_list = []
civ_div_dict = {}
for civ_span in civ_content_text_div.select(civ_selector):
    span_id = civ_span['id']
    if ('_' not in span_id) and (span_id != 'Sources'):
        civ_div_dict[span_id] = civ_span.parent.next_sibling.next_sibling
        civ_list.append(span_id)
store_objects(civ_list=civ_list)

Pickling to C:\Users\dev\Documents\repositories\notebooks\age-of-empires-ii\saves\pickle\civ_list.pickle


In [41]:

overview_selector = 'div[title="Overview"]'
rows_list = []
for civ_name in civ_list:
    civ_key_list = []
    for civ_b in civ_div_dict[civ_name].find_all('b'):
        civ_key = civ_b.string
        if civ_key.endswith(':'):
            civ_key_list.append(civ_key)
    overview_str = civ_div_dict[civ_name].select(overview_selector)[0].text
    overview_str = re.sub(r'\s+', ' ', overview_str).strip()
    key_regex = re.compile(' ({}) '.format('|'.join(civ_key_list)))
    feature_list = key_regex.split(overview_str)
    row_dict = {key[:-1]: value for key, value in zip(feature_list[1::2], feature_list[2::2])}
    row_dict[column_name] = civ_name
    rows_list.append(row_dict)
overview_df = pd.DataFrame(rows_list)
overview_df = overview_df.set_index(column_name)
overview_df.index.name = column_name

In [44]:

civilizations_df = pd.merge(left=civilizations_df, right=overview_df, how='outer', left_index=True,
                            right_index=True, suffixes=('_civilizations', '_overview'))

In [50]:

columns_list = ['Unique unit', 'Unique units']
unique_series = civilizations_df[columns_list].fillna(value='').apply(lambda x: ' '.join(x).strip(),
                                                                      axis=1)
civilizations_df['Unique units'] = unique_series

In [53]:

columns_list = ['Architectural Style', 'Wonder Name', 'Civilization bonuses',
                                    'Specialty', 'Team bonus', 'Unique technologies',
                                    'Unique units', 'Wonder']
civilizations_df = civilizations_df[columns_list]

In [55]:

civilizations_df.sample(5).T.sample(5).T

,Unique units,Specialty,Civilization bonuses,Wonder,Wonder Name
Civilization,,,,,
Ethiopians,Shotel Warrior,Archers and siege weapons,Archers fire +15% faster. Receive +100 food an...,Biete Medhane Alem,NaN
Malians,Gbeto,Infantry,Buildings cost -15% wood (except Farms). Barra...,Great Mosque of Djenné,NaN
Turks,Janissary,Gunpowder units,Gunpowder units have +25% HP. Gunpowder techno...,Selimiye Mosque,Selimiye Mosque
Incas,Kamayuk Slinger,Infantry,Start the game with an Eagle Scout. Start with...,Temple of the Sun at Macchu Picchu,NaN
Burmese,Arambai,Monks and elephants,Lumber Camp technologies are free. Infantry ha...,Shwezigon Pagoda,NaN


In [58]:

file_path = os.path.join(DATA_CSV_FOLDER, 'civilizations_df.csv')
print('Saving to {}'.format(os.path.abspath(file_path)))
civilizations_df.to_csv(file_path)
save_dataframes(civilizations_df=civilizations_df)
xlsx_folder = os.path.join(DATA_FOLDER, 'xlsx')
os.makedirs(name=xlsx_folder, exist_ok=True)
file_path = os.path.join(xlsx_folder, 'civilizations_df.xlsx')
print('Saving to {}'.format(os.path.abspath(file_path)))
civilizations_df.to_excel(file_path)

Saving to C:\Users\dev\Documents\repositories\notebooks\age-of-empires-ii\data\csv\civilizations_df.csv
Saving to C:\Users\dev\Documents\repositories\notebooks\age-of-empires-ii\saves\csv\civilizations_df.csv
Saving to C:\Users\dev\Documents\repositories\notebooks\age-of-empires-ii\data\xlsx\civilizations_df.xlsx
